In [1]:
from remote_sensing_ddpm.datasets.uc_merced_land_use.uc_merced_dataset import UCMerced

uc_merced_data_root = "/Users/alexanderlontke/Documents/Uni/St. Gallen/SS_23/master_thesis/data/UCMerced_LandUse/Images"

dataset = UCMerced(
    data_root=uc_merced_data_root,
    phase="train",
    split_ratios={"train": 1.0}
)

Loading Classes:: 100%|██████████| 21/21 [00:02<00:00,  8.33it/s]


In [2]:
count = 0
for i, entry in enumerate(dataset):
    if not entry["image"].shape == (3, 256, 256):
        print(entry["image"].shape)
        count +=1
print(count)

0


In [9]:
print(entry)

{'image': tensor([[[-0.7569, -0.6000, -0.5843,  ..., -0.1373, -0.2000, -0.5765],
         [-0.3882, -0.2471, -0.3647,  ..., -0.3647, -0.4196, -0.5922],
         [-0.3333, -0.2784, -0.4588,  ..., -0.7725, -0.7647, -0.7098],
         ...,
         [-0.0588, -0.2314, -0.4118,  ..., -0.0980, -0.6157, -0.4667],
         [ 0.0745, -0.2627, -0.6235,  ..., -0.0745, -0.4824, -0.2784],
         [ 0.1686, -0.1922, -0.6627,  ...,  0.0118, -0.1373, -0.2784]],

        [[-0.6392, -0.4902, -0.4588,  ...,  0.0824, -0.0039, -0.4275],
         [-0.2549, -0.0824, -0.2000,  ..., -0.1373, -0.2078, -0.4196],
         [-0.1843, -0.0824, -0.2706,  ..., -0.6314, -0.6314, -0.5686],
         ...,
         [-0.0431, -0.1843, -0.2941,  ...,  0.0353, -0.5294, -0.3647],
         [ 0.0510, -0.2627, -0.5686,  ...,  0.0510, -0.3647, -0.1451],
         [ 0.1216, -0.2157, -0.6471,  ...,  0.1686,  0.0118, -0.1216]],

        [[-0.6392, -0.5216, -0.4902,  ..., -0.0667, -0.1294, -0.5059],
         [-0.2706, -0.1529, -0.2706

In [3]:
def conv_output(h, kernel, stride):
    return (h-kernel)/stride +1

In [12]:
conv_output(256, 5, 1)

252.0

In [14]:
conv_output(252, 2, 2)

126.0

In [15]:
conv_output(126,5,1)

122.0

In [16]:
conv_output(122,2,2)

61.0